# Crypto SQL Market Tracker: SQL Demo and Analytics
This notebook connects to a local SQLite database containing Bitcoin & Ethereum price history.
We store the data, run SQL queries, convert results to DataFrames, and export to Excel for Tableau use.

Tools Demonstrated:
Python + SQLite + Pandas + SQL queries + Excel export

## Load CSV Data Into SQLite Database

This cell reads all raw crypto CSV files, extracts the important columns, assigns each asset a `coin` and `symbol` based on its filename, and saves everything into an SQLite database for analysis.

Details:
- Automatically detects timestamp and price columns
- Standardizes timestamps
- Labels assets using file names (ex: BTC, ETH)
- Skips duplicates when rerunning the script
- Stores all data in `CryptoData.db` under the `crypto_prices` table


In [ ]:
```python
import sqlite3
import pandas as pd
import glob
import os
from pathlib import Path
from datetime import datetime

# File paths
DatabasePath = "CryptoData.db"
RawDataPath = "data/raw/*.csv"

# Create database connection
connect = sqlite3.connect(DatabasePath)
cursor = connect.cursor()

# Create SQL table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS crypto_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    coin TEXT,
    symbol TEXT,
    timestamp TEXT UNIQUE,
    price REAL,
    market_cap REAL,
    volume REAL
);
""")

# Grab all CSV files in data/raw
files = glob.glob(RawDataPath)

# Loop through each CSV
for file in files:
    print(f"Loading {file}...")

    df = pd.read_csv(file)

    # Make column names lowercase for easier matching
    df.columns = df.columns.str.lower()

    # Detect and format timestamp column safely
    time_cols = [col for col in df.columns if "time" in col or "date" in col]
    if time_cols:
        df["timestamp"] = pd.to_datetime(df[time_cols[0]], errors="coerce").astype(str)
    else:
        print("No timestamp column found, skipping file.")
        continue

    # Detect price column safely
    price_cols = [col for col in df.columns if "price" in col or "close" in col]
    if price_cols:
        df["price"] = pd.to_numeric(df[price_cols[0]], errors="coerce")
    else:
        print("No price column found, skipping file.")
        continue

    # Optional columns (market cap & volume)
    df["market_cap"] = pd.to_numeric(df.get("market_cap", 0), errors="coerce")
    df["volume"] = pd.to_numeric(df.get("volume", 0), errors="coerce")

    # Tag coin & symbol based on filename
    name = os.path.basename(file).lower()
    if "bitcoin" in name or "btc" in name:
        df["coin"], df["symbol"] = "Bitcoin", "BTC"
    elif "ethereum" in name or "eth" in name:
        df["coin"], df["symbol"] = "Ethereum", "ETH"
    else:
        df["coin"] = name.split(".")[0]
        df["symbol"] = None

    # Insert into SQL database using insert-ignore to avoid duplicates
    for _, row in df.iterrows():
        cursor.execute("""
        INSERT OR IGNORE INTO crypto_prices (coin, symbol, timestamp, price, market_cap, volume)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (row.coin, row.symbol, row.timestamp, row.price, row.market_cap, row.volume))

# Commit and close
connect.commit()
connect.close()

print("All data loaded into SQLite database")
'''



## Verify Database Upload

Now that the data is inserted, this section checks:

- How many records exist for each asset
- Whether the table structure looks correct
- A small preview of stored rows to confirm nothing looks off

This ensures the database loaded properly before doing analysis or visualizations.


```python
import sqlite3
import pandas as pd

# Connect to database
connect = sqlite3.connect("CryptoData.db")

# Show count of records per coin
query = """
SELECT coin, symbol, COUNT(*) AS total_rows
FROM crypto_prices
GROUP BY coin, symbol
ORDER BY total_rows DESC;
"""

df_counts = pd.read_sql(query, connect)
display(df_counts)

# Preview first 5 rows in table
df_preview = pd.read_sql("SELECT * FROM crypto_prices LIMIT 5;", connect)
display(df_preview)

connect.close()
'''


## Visualize Price Trends

This section pulls price history from the database and plots a time-series chart for a selected asset.  
Useful for spotting overall trends, volatility, or any missing/broken date ranges in the data.

You can change the ticker below (BTC, ETH, etc.) to view different assets.


## Creating a daily summary table

The goal here is to collapse the raw price data into one row per coin per day.

This gives us:
- open, close, high, low for the day
- total volume
- percent change from the previous day

Keeping everything in SQL makes it easy to reuse later for dashboards or querying.
